<a href="https://colab.research.google.com/github/k10sj02/tidy-pydy-tuesday/blob/main/2025_01_21_himalayan_expeditions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpledbf pandas pyreadstat dbfread

In [ ]:
!wget --user-agent="Mozilla/5.0" -O himalaya.zip "https://www.himalayandatabase.com/downloads/Himalayan%20Database.zip"

--2025-11-01 01:59:10--  https://www.himalayandatabase.com/downloads/Himalayan%20Database.zip
Resolving www.himalayandatabase.com (www.himalayandatabase.com)... 66.96.146.129
Connecting to www.himalayandatabase.com (www.himalayandatabase.com)|66.96.146.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42737250 (41M) [application/zip]
Saving to: ‘himalaya.zip’

himalaya.zip        100%[===================>]  40.76M  35.1MB/s    in 1.2s    

2025-11-01 01:59:11 (35.1 MB/s) - ‘himalaya.zip’ saved [42737250/42737250]



In [ ]:
import zipfile

extract_dir = tempfile.mkdtemp()

with zipfile.ZipFile("himalaya.zip", "r") as z:
    z.extractall(extract_dir)

extract_dir

'/tmp/tmpkom07mot'

In [ ]:
import glob

peaks_dbf = glob.glob(os.path.join(extract_dir, "**", "peaks.DBF"), recursive=True)[0]
exped_dbf = glob.glob(os.path.join(extract_dir, "**", "exped.DBF"), recursive=True)[0]

peaks_dbf, exped_dbf

('/tmp/tmpkom07mot/Himalayan Database/HIMDATA/peaks.DBF',
 '/tmp/tmpkom07mot/Himalayan Database/HIMDATA/exped.DBF')

In [ ]:
from dbfread import DBF
import pandas as pd

peaks_records = list(DBF(peaks_dbf, load=True, char_decode_errors='ignore'))
exped_records = list(DBF(exped_dbf, load=True, char_decode_errors='ignore'))

peaks_temp = pd.DataFrame(peaks_records)
exped_temp = pd.DataFrame(exped_records)

In [ ]:
himal_map = {
    0: "Unclassified",
    1: "Annapurna",
    2: "Api/Byas Risi/Guras",
    3: "Damodar",
    4: "Dhaulagiri",
    5: "Ganesh/Shringi",
    6: "Janak/Ohmi Kangri",
    7: "Jongsang",
    8: "Jugal",
    9: "Kangchenjunga/Simhalila",
    10: "Kanjiroba",
    11: "Kanti/Palchung",
    12: "Khumbu",
    13: "Langtang",
    14: "Makalu",
    15: "Manaslu/Mansiri",
    16: "Mukut/Mustang",
    17: "Nalakankar/Chandi/Changla",
    18: "Peri",
    19: "Rolwaling",
    20: "Saipal"
}

peaks_temp["HIMAL_FACTOR"] = peaks_temp["HIMAL"].map(himal_map).fillna("Unknown")


In [ ]:
region_map = {
    0: "Unclassified",
    1: "Kangchenjunga-Janak",
    2: "Khumbu-Rolwaling-Makalu",
    3: "Langtang-Jugal",
    4: "Manaslu-Ganesh",
    5: "Annapurna-Damodar-Peri",
    6: "Dhaulagiri-Mukut",
    7: "Kanjiroba-Far West"
}

peaks_temp["REGION_FACTOR"] = peaks_temp["REGION"].map(region_map).fillna("Unknown")


In [ ]:
phost_map = {
    0: "Unclassified",
    1: "Nepal only",
    2: "China only",
    3: "India only",
    4: "Nepal & China",
    5: "Nepal & India",
    6: "Nepal, China & India"
}

peaks_temp["PHOST_FACTOR"] = peaks_temp["PHOST"].map(phost_map).fillna("Unknown")


In [ ]:
pstatus_map = {
    0: "Unknown",
    1: "Unclimbed",
    2: "Climbed"
}

peaks_temp["PSTATUS_FACTOR"] = peaks_temp["PSTATUS"].map(pstatus_map).fillna("Invalid")


In [ ]:
season_map = {
    0: "Unknown",
    1: "Spring",
    2: "Summer",
    3: "Autumn",
    4: "Winter"
}

exped_temp["SEASON_FACTOR"] = exped_temp["SEASON"].map(season_map)


In [ ]:
host_map = {
    0: "Unknown",
    1: "Nepal",
    2: "China",
    3: "India"
}

exped_temp["HOST_FACTOR"] = exped_temp["HOST"].map(host_map)


In [ ]:
termreason_map = {
    0: "Unknown",
    1: "Success (main peak)",
    2: "Success (subpeak, foresummit)",
    3: "Success (claimed)",
    4: "Bad weather (storms, high winds)",
    5: "Bad conditions (deep snow, avalanching, falling ice, or rock)",
    6: "Accident (death or serious injury)",
    7: "Illness, AMS, exhaustion, or frostbite",
    8: "Lack (or loss) of supplies, support or equipment",
    9: "Lack of time",
    10: "Route technically too difficult, lack of experience, strength, or motivation",
    11: "Did not reach base camp",
    12: "Did not attempt climb",
    13: "Attempt rumored",
    14: "Other"
}

exped_temp["TERMREASON_FACTOR"] = exped_temp["TERMREASON"].map(termreason_map)


In [ ]:
exped_tidy = exped_temp[exped_temp["YEAR"].astype(str).str.contains("202[0-4]")]
peaks_tidy = peaks_temp.copy()

In [ ]:
peaks_tidy
exped_tidy

,EXPID,PEAKID,YEAR,SEASON,HOST,ROUTE1,ROUTE2,ROUTE3,ROUTE4,NATION,...,COMRTE,STDRTE,PRIMRTE,PRIMMEM,PRIMREF,PRIMID,CHKSUM,SEASON_FACTOR,HOST_FACTOR,TERMREASON_FACTOR
10416,EVER20101,EVER,2020,1,2,N Col-NE Ridge,,,,China,...,True,True,False,False,False,,2465291,Spring,China,Success (main peak)
10417,EVER20102,EVER,2020,1,2,N Col-NE Ridge,,,,China,...,True,True,False,False,False,,2465292,Spring,China,Success (main peak)
10418,EVER20103,EVER,2020,1,2,N Col-NE Ridge,,,,China,...,True,True,False,False,False,,2465293,Spring,China,Success (main peak)
10515,AMAD20301,AMAD,2020,3,1,SW Ridge,,,,Nepal,...,True,False,False,False,False,,2463299,Autumn,Nepal,Success (main peak)
10516,AMAD20302,AMAD,2020,3,1,SW Ridge,,,,USA,...,True,False,False,False,False,,2463299,Autumn,Nepal,Success (main peak)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11570,TKRE24301,TKRE,2024,3,1,E Face,,,,UK,...,False,False,False,False,False,,2465925,Autumn,Nepal,Success (main peak)
11571,TUKU24301,TUKU,2024,3,1,,,,,Italy,...,False,False,False,False,False,,5309,Autumn,Nepal,"Bad conditions (deep snow, avalanching, fallin..."
11575,YASA21301,YASA,2021,3,1,S Pillar (to 6000m),SE Ridge (to 6000m),,,Italy,...,False,False,False,False,True,TKPO21302,2464682,Autumn,Nepal,Other
11576,YASA22301,YASA,2022,3,1,"W Face-SW Ridge (up), E Ridge (down)",,,,Slovenia,...,False,False,False,False,False,,2465094,Autumn,Nepal,Success (main peak)


In [ ]:
with pd.option_context('display.max_columns', None):
  display(peaks_tidy.head())

,PEAKID,PKNAME,PKNAME2,LOCATION,HEIGHTM,HEIGHTF,HIMAL,REGION,OPEN,UNLISTED,TREKKING,TREKYEAR,RESTRICT,PHOST,PSTATUS,PEAKMEMO,PYEAR,PSEASON,PEXPID,PSMTDATE,PCOUNTRY,PSUMMITERS,PSMTNOTE,REFERMEMO,PHOTOMEMO,HIMAL_FACTOR,REGION_FACTOR,PHOST_FACTOR,PSTATUS_FACTOR
0,AMAD,Ama Dablam,Amai Dablang,Khumbu Himal,6814,22356,12,2,True,False,False,,,1,2,"Other map altitudes:\r\n 6814m - HMG-MT, HMG...",1961,1,AMAD61101,Mar 13,"New Zealand, USA, UK","Mike Gill, Wally Romanes, Barry Bishop, Michae...",,None,W Face (High 126:5 May 1993)\r\nSE Face (High ...,Khumbu,Khumbu-Rolwaling-Makalu,Nepal only,Climbed
1,AMPG,Amphu Gyabjen,Amphu Gyabien,Khumbu Himal (N of Ama Dablam),5630,18471,12,2,True,False,False,,Opened in 2002,1,2,"Other map altitudes:\r\n 5630m - HMG-Finn, N...",1953,1,AMPG53101,Apr 11,UK,"John Hunt, Tom Bourdillon",,None,None,Khumbu,Khumbu-Rolwaling-Makalu,Nepal only,Climbed
2,ANN1,Annapurna I,,Annapurna Himal,8091,26545,1,5,True,False,False,,,1,2,"Other map altitudes:\r\n 8091m - HMG-MT, HMG...",1950,1,ANN150101,Jun 03,France,"Maurice Herzog, Louis Lachenal",,Dyhrenfurth history 1950-1977 (MM 58:44-47 Nov...,S Face (High 122:3 Jan 1993) (Beghin accident)...,Annapurna,Annapurna-Damodar-Peri,Nepal only,Climbed
3,ANN2,Annapurna II,,Annapurna Himal,7937,26040,1,5,True,False,False,,,1,2,"Other map altitudes:\r\n 7937m - HMG-MT, HMG...",1960,1,ANN260101,May 17,"UK, Nepal","Richard Grant, Chris Bonington, Ang Nyima Sherpa",,Dyhrenfurth history 1960-1976 (MM 51:36-37 Sep...,N Face (MM 51:36 Sep 1976),Annapurna,Annapurna-Damodar-Peri,Nepal only,Climbed
4,ANN3,Annapurna III,,Annapurna Himal,7555,24787,1,5,True,False,False,,,1,2,"Other map altitudes:\r\n 7555m - HMG-MT, HMG...",1961,1,ANN361101,May 06,India,"Mohan S. Kohli, Sonam Gyatso, Sonam Girmi",,None,S Side (MM 125:11 Jan 1989)\r\nSW Face (MM 71:...,Annapurna,Annapurna-Damodar-Peri,Nepal only,Climbed


In [ ]:
with pd.option_context('display.max_columns', None):
  display(exped_tidy.head())

,EXPID,PEAKID,YEAR,SEASON,HOST,ROUTE1,ROUTE2,ROUTE3,ROUTE4,NATION,LEADERS,SPONSOR,SUCCESS1,SUCCESS2,SUCCESS3,SUCCESS4,ASCENT1,ASCENT2,ASCENT3,ASCENT4,CLAIMED,DISPUTED,COUNTRIES,APPROACH,BCDATE,SMTDATE,SMTTIME,SMTDAYS,TOTDAYS,TERMDATE,TERMREASON,TERMNOTE,HIGHPOINT,TRAVERSE,SKI,PARAPENTE,CAMPS,ROPE,TOTMEMBERS,SMTMEMBERS,MDEATHS,TOTHIRED,SMTHIRED,HDEATHS,NOHIRED,O2USED,O2NONE,O2CLIMB,O2DESCENT,O2SLEEP,O2MEDICAL,O2TAKEN,O2UNKWN,OTHERSMTS,CAMPSITES,ROUTEMEMO,ACCIDENTS,ACHIEVMENT,AGENCY,COMRTE,STDRTE,PRIMRTE,PRIMMEM,PRIMREF,PRIMID,CHKSUM,SEASON_FACTOR,HOST_FACTOR,TERMREASON_FACTOR
10416,EVER20101,EVER,2020,1,2,N Col-NE Ridge,,,,China,Tibetan Rope-Fixing,Tibetan Rope-Fixing Everest North 2020,True,False,False,False,,,,,False,False,,Lhasa->Tingri->Everest BC,None,2020-05-26,1515,0,0,None,1,,8849,False,False,False,3,0,0,0,0,6,6,0,False,True,False,True,False,True,False,False,False,,"BC,ABC,C1,C2,C3,Smt(26/05)",None,,,Holy Mountain Adventure,True,True,False,False,False,,2465291,Spring,China,Success (main peak)
10417,EVER20102,EVER,2020,1,2,N Col-NE Ridge,,,,China,Ci Luo (Tselo),Chinese Mount Everest Survey Team,True,False,False,False,,,,,False,False,,,None,2020-05-27,0945,0,0,None,1,,8849,False,False,False,3,0,12,8,0,0,0,0,False,True,False,True,False,True,False,False,False,,"BC,ABC,C1,C2,C3,Smt(27/05)",From www.mt8848.com:\r\n\r\nThe 2020 Mount Eve...,,,,True,True,False,False,False,,2465292,Spring,China,Success (main peak)
10418,EVER20103,EVER,2020,1,2,N Col-NE Ridge,,,,China,Tsering Samdrup,Holy Mountain Adventure Everest Expedition 2020,True,False,False,False,,,,,False,False,,Lhasa->Tingri->Everest BC,2020-04-23,2020-05-28,0545,35,38,2020-05-31,1,,8849,False,False,False,3,0,20,14,0,22,21,0,False,True,False,True,False,True,False,False,False,,"BC(23/04,5200m),IC(26/04,5800m),ABC(05/01,6500...","Expedition Itinerary:\r\n\r\n23 Apr, arrived a...",,,Holy Mountain Adventure,True,True,False,False,False,,2465293,Spring,China,Success (main peak)
10515,AMAD20301,AMAD,2020,3,1,SW Ridge,,,,Nepal,Chhang Dawa Sherpa,Seven Summit Treks Ama Dablam Expedition 2020,True,False,False,False,,,,,False,False,"Canada, Czech Republic, France, Poland, Russia...",,2020-11-09,2020-11-10,1300,1,0,None,1,,6814,False,False,False,2,0,14,9,0,19,14,0,False,False,True,False,False,False,False,False,False,,"BC(09/11,4450m),C1(5600m),C2(5900m),Smt(10,12-...",None,,,Seven Summit Treks,True,False,False,False,False,,2463299,Autumn,Nepal,Success (main peak)
10516,AMAD20302,AMAD,2020,3,1,SW Ridge,,,,USA,Garrett Madison,Madison Mountaineering Ama Dablam Expedition 2020,True,False,False,False,,,,,False,False,"Canada, Qatar",Lukla->Pangboche->Ama Dablam BC,2020-11-01,2020-11-10,1300,9,11,2020-11-12,1,,6814,False,False,False,2,0,6,6,0,8,8,0,False,False,True,False,False,False,False,False,False,,"BC(01/11,4700m),C1(04/11,5600m),C2(10/11,6000m...","On summit day, team left high camp around 1:30...",,,Himalayan Guides,True,False,False,False,False,,2463299,Autumn,Nepal,Success (main peak)


In [ ]:
# 1. What is the distribution of climbing status (PSTATUS) across the different mountain ranges (HIMAL_FACTOR)?

print(peaks_tidy['PSTATUS'].unique())
# PSTATUS appears to be a categorical/binary variable (1, 2).
# I need to check the documentation to confirm what each code represents
# (e.g., success vs. failure, or other status categories).

print(peaks_tidy['HIMAL_FACTOR'].unique())
# HIMAL_FACTOR contains the names of mountain ranges.
# Some names include slashes because they refer to combined or adjacent sub-ranges.

# Next step: group PSTATUS by HIMAL_FACTOR to see how climbing status varies
# across mountain ranges (e.g., using a crosstab or histogram).

[2 1]
['Khumbu' 'Annapurna' 'Api/Byas Risi/Guras' 'Manaslu/Mansiri' 'Langtang'
 'Damodar' 'Nalakankar/Chandi/Changla' 'Ganesh/Shringi' 'Peri' 'Rolwaling'
 'Dhaulagiri' 'Kangchenjunga/Simhalila' 'Mukut/Mustang' 'Jugal' 'Jongsang'
 'Saipal' 'Kanjiroba' 'Makalu' 'Janak/Ohmi Kangri' 'Kanti/Palchung']


In [ ]:
table = pd.crosstab(
    peaks_tidy['HIMAL_FACTOR'], # index: unique values become the ROWS of the table
    peaks_tidy['PSTATUS'], # columns: unique values become the COLUMNS of the table
    normalize='index').mul(100).round(2)

table.sort_values(by=1, ascending=False)

PSTATUS,1,2
HIMAL_FACTOR,,
Kanti/Palchung,66.67,33.33
Api/Byas Risi/Guras,61.11,38.89
Saipal,50.00,50.00
Janak/Ohmi Kangri,46.67,53.33
Ganesh/Shringi,45.00,55.00
Makalu,33.33,66.67
Nalakankar/Chandi/Changla,29.41,70.59
Jongsang,28.57,71.43
Kangchenjunga/Simhalila,26.67,73.33


In [ ]:
# 2. Which mountain range (HIMAL_FACTOR) has the highest average peak height (HEIGHTM)?
# 3. What is the distribution of peak heights (HEIGHTM) for peaks that are open (OPEN) versus not open?
# 4. Which climbing routes (ROUTE1, ROUTE2, ROUTE3, ROUTE4) have the highest success rates (SUCCESS1, SUCCESS2, SUCCESS3, SUCCESS4) across all expeditions?
# 5. How does the use of supplemental oxygen (O2USED, O2NONE) affect summit success rates?
# 6. How often does bad weather (TERMREASON = 4) cause termination compared to technical difficulty (TERMREASON = 10)?
# 7. Are expeditions with no hired personnel (NOHIRED) associated with higher or lower death rates?